In [ ]:
!for i in $(seq -f "%02g" 1 10); do cp scripts_2d/train_${i}.sh scripts_2d/eval_${i}.sh; done

In [ ]:
!for i in $(seq -f "%02g" 1 10); do cp scripts/train_${i}.sh scripts/eval_${i}.sh; done

In [5]:
import os
import re
from glob import glob

for scripts_dir in ["scripts_2d", "scripts"]:
    for i in range(1, 11):
        script_path = f"{scripts_dir}/eval_{i:02d}.sh"
        out_paths = list(glob(f"task{i:02d}{('_2d' if '2d' in scripts_dir else '')}_*.out"))
        if "2d" not in script_path:
            out_paths = [x for x in out_paths if "2d" not in x]
        if len(out_paths) == 0:
            print(f"WARNING!!!! out file for {script_path} not found")
            continue
        else:
            valid_out_paths = []
            for out_path in out_paths:
                out = open(out_path).read()
                if "Run summary:" in out:
                    found = re.findall("https://wandb.ai/yuhuiz/MedicalSegmentation/runs/(.*)", out)
                    if len(set(found)) == 1:
                        run_id = found[0]
                        ckpt_paths = list(glob(f"MedicalSegmentation/{run_id}/checkpoints/*.ckpt"))
                        if len(ckpt_paths) == 1:
                            valid_out_paths.append([out_path, ckpt_paths[0]])
            if len(valid_out_paths) == 0:
                print(f"WARNING!!!! out file for {script_path} not found")
                continue
            elif len(valid_out_paths) > 1:
                print(f"ERROR!!! more than 1 out file found for {script_path}: {valid_out_paths}")
            else:
                out_path, ckpt_path = valid_out_paths[0]
                eval_cmd = open(script_path).read().strip()
                eval_cmd += f" \\\n  --evaluation 1 \\\n  --model_path {ckpt_path}\n"
                with open(script_path, "w") as fout:
                    fout.write(eval_cmd)

WARNING!!!! out file for scripts_2d/eval_08.sh not found
WARNING!!!! out file for scripts/eval_03.sh not found
WARNING!!!! out file for scripts/eval_07.sh not found
